# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [28]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

response = requests.get(url, time.sleep(5))
html_content = response.text
html_content

soup = BeautifulSoup(html_content, 'html.parser')
soup

tablas = soup.find_all('table')
quarterly_table = tables[0]
rows = quarterly_table.find_all('tr')
# Crear listas para almacenar los datos data = [] for row in rows: cols = row.find_all('td') cols = [col.text.strip() for col in cols] data.append(cols) # Crear un DataFrame con los datos extraídos df = pd.DataFrame(data) # Mostrar el DataFrame print(df)



NameError: name 'tables' is not defined